In [71]:
import pyprob
from pyprob import Model
from pyprob.distributions import Normal, Mixture

import torch
import numpy as np

%matplotlib inline

# Inference compilation on 1D 2-cluster GMM

Consider a 1-dimensional Gaussian Mixture model where:
 * Number of components $K$ fixed
 * Component means $\mu_i \sim N(0, 100)$ and standard deviation $\sigma_i = 0.1$ for all $i \in [K]$
 * For each observation $\texttt{obs}_i$ $(i \in [\texttt{num_obs}])$:
     * Component membership $c_i \sim \text{Unif}([K])$
     * Conditional distribution $\texttt{obs}_i \mid c_i = j \sim N(\mu_j, 0.1)$.

In [85]:
class GMM(Model):
    def __init__(self, num_obs, K):
        super().__init__(name='Gaussian mixture model')
        self.K = K
        self.num_obs = num_obs

    def forward(self):
        mus = [pyprob.sample(Normal(0, 100)) for _ in range(self.K)]
        probs = [1./self.K for _ in range(self.K)]
        likelihood = Mixture(
            [Normal(mus[i], 0.1) for i in range(self.K)],
            probs=probs)
        
        for i in range(self.num_obs):
            pyprob.observe(likelihood, name=f'obs{i}')

        return torch.tensor(mus)

Note that the number of observations is fixed during model specification and must be the same during inference. This is a current limitation of inference compilation in PyProb.

Let's instantiate this model with $K=2$ components and $4$ data points, and then lets sample the prior and compute an empirical mean:

In [89]:
model = GMM(num_obs=4, K=2)
prior = model.prior_results(num_traces=1000)
prior.expectation(lambda x: x)

Time spent  | Time remain.| Progress             | Trace     | Traces/sec
0d:00:00:00 | 0d:00:00:00 | #################### | 1000/1000 | 2,475.39       


tensor([2.7497, 1.6490])

We see that the empirical mean is close to the true mean of the prior $N(0, 100)$ over $\mu_i$s.

Next, let's demonstrate how we can perform inference compilation. In the following snippet, we:
 * Sample the model for 20000 execution traces
 * Use a feedforward neural network to embed it into a 16-dimensional vector
 * Train a LSTM + FFW NN + Random Variable specific proposal distribution to maximize the probability of the sample traces
 
This trained network is referred to as the "inference compilation artifact," and is used during inference as an importance sampling proposer which accounts for the state present up until this point within an execution trace.

In [68]:
model.learn_inference_network(
        num_traces=20000,
        observe_embeddings={f'obs{i}x' : {'dim' : 16} for i in range(model.num_obs)},
        inference_network=pyprob.InferenceNetwork.LSTM)

Creating new inference network...
Observable obs0x: reshape not specified, using shape torch.Size([]).
Observable obs0x: using embedding dim torch.Size([16]).
Observable obs0x: observe embedding not specified, using the default FEEDFORWARD.
Observable obs0x: embedding depth not specified, using the default 2.
Observable obs1x: reshape not specified, using shape torch.Size([]).
Observable obs1x: using embedding dim torch.Size([16]).
Observable obs1x: observe embedding not specified, using the default FEEDFORWARD.
Observable obs1x: embedding depth not specified, using the default 2.
Observable obs2x: reshape not specified, using shape torch.Size([]).
Observable obs2x: using embedding dim torch.Size([16]).
Observable obs2x: observe embedding not specified, using the default FEEDFORWARD.
Observable obs2x: embedding depth not specified, using the default 2.
Observable obs3x: reshape not specified, using shape torch.Size([]).
Observable obs3x: using embedding dim torch.Size([16]).
Observable

After training the inference network, we can use it to perform posterior importance sampling. Below, we condition on some observed data, draw 200 importance samples using our trained inference network, and finally compute a posterior mean.

In [70]:
observed_list = [
    100,
    -100,
    100,
    -100,
]

posterior = model.posterior_results(
    num_traces=200,
    inference_engine=pyprob.InferenceEngine.IMPORTANCE_SAMPLING_WITH_INFERENCE_NETWORK, # specify which inference engine to use
    observe={
        **{f'obs{i}': observed_list[i] for i in range(model.num_obs)},
    })
print(posterior.expectation(lambda x: x))

Time spent  | Time remain.| Progress             | Trace   | Traces/sec
0d:00:00:01 | 0d:00:00:00 | #################### | 200/200 | 156.79       
tensor([[ 99.2688],
        [-99.9758]])


Notice that the posterior means $\{\mu_0, \mu_0\} \approx \{100, -100\}$. Because our prior on component means $\mu_i$ is quite diffuse (with standard deviation $100$), we expect the posterior to be quite close in shape to the likelihood. Since our data is exactly at $\pm 100$, these results make sense and suggest IC is working as expected.